# {The Rising Impact of Data Breaches in the U.S.}📝

![Banner](./assets/banner.jpeg)

## Topic
*What problem are you (or your stakeholder) trying to address?*
📝 <!-- Answer Below -->
I want to talk about cybersecurity and data privacy, specifically analyzing patterns in data 
breaches in the United States. Every year, millions of people are affected by compromised personal 
data, which can result in identity theft, financial loss, and a decline in faith in businesses and 
technology. The rise of cloud platforms, artificial intelligence, and digitalservices has led to a 
rise in the collection of sensitive data, but laws and safeguards are frequently lagging behind. 
This makes the topic important today. Analyzing breach trends can help us identify weak points and 
improve data security for both individuals and corporations.

## Project Question
*What specific question are you seeking to answer with this project?*
*This is not the same as the questions you ask to limit the scope of the project.*
📝 <!-- Answer Below -->
- How has breach frequency changed over time in the U.S.?
- Which industries are most frequently targeted?
- What types of data (PII, PHI, financial) are most often exposed?
- Can a model predict whether a breach contains PII/PHI based on the breach summary?

## What would an answer look like?
*What is your hypothesized answer to your question?*
📝 <!-- Answer Below -->
- The number of reported breaches increases over time (more reporting & attacks)
- Finance and healthcare will be among the most-targeted industries
- PII and health data will appear frequently
- A text classifier can detect PII mentions in breach summaries with reasonable accuracy

## Data Sources
*What 3 data sources have you identified for this project?*
*How are you going to relate these datasets?*
📝 <!-- Answer Below -->
Data Sources Used:
1. Dataset 1 (Df_1.csv)
2. Dataset 2 (breach_report.csv)
3. Dataset 3 (cyber_security_breaches.csv)
By finding the common fields that show up in all of the datasets—such as the breach date, organization name, industry sector, breach type, and the quantity of documents exposed—I am connecting them. These shared fields enable me to combine each dataset into a single, cohesive dataset, despite the fact that they each concentrate on distinct facets of cybersecurity incidents. Even if datasets cannot be directly combined, I can still conduct independent analyses and use the findings to bolster the main conclusions. I can analyze trends across industries, time periods, and breach characteristics by merging the data in this way, giving me a broader and more comprehensive picture of U.S. data breaches.



## Approach and Analysis
*What is your approach to answering your project question?*
*How will you use the identified data to answer your project question?*
My method involves cleaning and standardizing all of the breach data so that it can be studied uniformly. After that, I examine the data to find trends, outliers, and connections between factors like year, industry, type of breach, and records exposed. After that, I make visualizations that show trends, such as how the frequency of breaches varies over time or which industries are most impacted. In order to determine whether breach severity can be anticipated using the dataset's attributes, I also employ machine-learning algorithms. Overall, I can address my project issue and demonstrate how data breaches have changed and what aspects are most important in security events by combining the datasets, investigating the trends, creating visuals, and using ML models.

I'll measure the frequency of breaches, the industries most affected, and the kinds of data exposed using the aggregated breach datasets. I can spot trends like rising breach frequency over time or industries with the highest occurrence rates by examining the dates, industry classifications, breach kinds, and record counts. While summary data can draw attention to significant changes or anomalous occurrences, visualizations will aid in the clear revelation of these trends. In order to determine whether particular characteristics, such as industry or breach kind, have a significant impact, I will also utilize the data to train machine-learning models that aim to forecast breach severity. The datasets collectively offer the proof required to address my project's questions and bolster significant findings regarding cybersecurity and data-privacy threats in the US.
📝 <!-- Start Discussing the project here; you can add as many code cells as you need -->



## Resources and References
*What resources and references have you used for this project?*
📝 <!-- Answer Below -->
https://www.nist.gov/cyberframework
https://ocrportal.hhs.gov/ocr/breach/breach_report.jsf
https://privacyrights.org/
https://www.idtheftcenter.org/
https://www.cisa.gov/
https://pandas.pydata.org/docs/
https://ocrportal.hhs.gov/ocr/breach/breach_report.jsf
https://privacyrights.org/data-breaches
https://www.idtheftcenter.org/

In [ ]:
# ⚠️ Make sure you run this cell at the end of your notebook before every submission!
!jupyter nbconvert --to python source.ipynb



[NbConvertApp] Converting notebook source.ipynb to python
[NbConvertApp] Writing 1271 bytes to source.py
